# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [20]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
#
import sqlite3
import pandas as pd
import numpy as np

ignore_cols = ['company_permalink','company_state_code','company_region','investor_permalink','investor_state_code','investor_region','funded_at','funded_year']
chunk_iter = pd.read_csv("https://bit.ly/3BPcobU", encoding="ISO-8859-1",chunksize = 5000, usecols=lambda col: col not in ignore_cols)

chunk_counter = 1
#chunk_lists = []
memory_footprints = list()
for chunk in chunk_iter:
  chunk_null_counts = chunk.isnull().sum()
  chunk_memory = chunk.memory_usage(deep=True).sum()/(1024*1024)
  memory_footprints.append(chunk_memory)

  print(f"Chunk {chunk_counter} details")
  print(f'\tMissing Value Counts: \n{chunk_null_counts}')
  print(f'\tChunk Memory: {chunk_memory} MB')
  print("\n\n")
  #chunk_lists.append(chunk)
  chunk_counter += 1
print(f'\tTotal Memory: {sum(memory_footprints)}')
#investments_df = pd.concat(chunk_lists)

Chunk 1 details
	Missing Value Counts: 
company_name                 0
company_category_code       52
company_country_code         0
company_city                64
investor_name                0
investor_category_code    2557
investor_country_code      778
investor_city              900
funding_round_type           0
funded_month                 0
funded_quarter               0
raised_amount_usd          653
dtype: int64
	Chunk Memory: 3.3087234497070312 MB



Chunk 2 details
	Missing Value Counts: 
company_name                 0
company_category_code       51
company_country_code         0
company_city                45
investor_name                0
investor_category_code    5000
investor_country_code      261
investor_city              313
funding_round_type           0
funded_month                 0
funded_quarter               0
raised_amount_usd          239
dtype: int64
	Chunk Memory: 3.1764907836914062 MB



Chunk 3 details
	Missing Value Counts: 
company_name                 0

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [21]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
#
ignore_cols = ['company_permalink','company_state_code','company_region','investor_permalink','investor_state_code','investor_region','funded_at','funded_year']
chunk_iter = pd.read_csv("https://bit.ly/3BPcobU", encoding="ISO-8859-1",chunksize = 5000, usecols=lambda col: col not in ignore_cols)

chunk_counter = 1
#chunk_lists = []
memory_footprints = list()
for chunk in chunk_iter:
  chunk_null_counts = chunk.isnull().sum()

  str_type_cols = chunk.select_dtypes(include=['object']).columns
  str_col_uniq_values = {}
  str_col_uniq_lt_50 = list()
  for col in str_type_cols:
    num_unique_values = len(chunk[col].unique())
    str_col_uniq_values[col] = num_unique_values

    num_total_values = len(chunk[col])
    if num_unique_values / num_total_values < 0.5:
      str_col_uniq_lt_50.append(col)

  for col in str_col_uniq_lt_50:
    chunk[col] = chunk[col].astype('category')

  chunk_memory = chunk.memory_usage(deep=True).sum()/(1024*1024)
  memory_footprints.append(chunk_memory)

  print(f"Chunk {chunk_counter} details")
  print(f'\tMissing Value Counts: \n{chunk_null_counts}\n')
  print(f'\tColumn Types: \n{chunk.dtypes}\n')
  print(f'\tChunk Memory: {chunk_memory} MB')
  print("\n\n")
  #chunk_lists.append(chunk)
  chunk_counter += 1
print(f'\tTotal Memory: {sum(memory_footprints)}')
#investments_df = pd.concat(chunk_lists)

Chunk 1 details
	Missing Value Counts: 
company_name                 0
company_category_code       52
company_country_code         0
company_city                64
investor_name                0
investor_category_code    2557
investor_country_code      778
investor_city              900
funding_round_type           0
funded_month                 0
funded_quarter               0
raised_amount_usd          653
dtype: int64

	Column Types: 
company_name                object
company_category_code     category
company_country_code      category
company_city              category
investor_name             category
investor_category_code    category
investor_country_code     category
investor_city             category
funding_round_type        category
funded_month              category
funded_quarter            category
raised_amount_usd          float64
dtype: object

	Chunk Memory: 0.6804780960083008 MB



Chunk 2 details
	Missing Value Counts: 
company_name                 0
company_cate

## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [23]:
# Your code goes here
conn = sqlite3.connect('crunchbase.db')
ignore_cols = ['company_permalink','company_state_code','company_region','investor_permalink','investor_state_code','investor_region','funded_at','funded_year']
chunk_iter = pd.read_csv("https://bit.ly/3BPcobU", encoding="ISO-8859-1",chunksize = 5000, usecols=lambda col: col not in ignore_cols)

chunk_counter = 1
#chunk_lists = []
memory_footprints = list()
for chunk in chunk_iter:
  chunk_null_counts = chunk.isnull().sum()

  str_type_cols = chunk.select_dtypes(include=['object']).columns
  str_col_uniq_values = {}
  str_col_uniq_lt_50 = list()
  for col in str_type_cols:
    num_unique_values = len(chunk[col].unique())
    str_col_uniq_values[col] = num_unique_values

    num_total_values = len(chunk[col])
    if num_unique_values / num_total_values < 0.5:
      str_col_uniq_lt_50.append(col)

  for col in str_col_uniq_lt_50:
    chunk[col] = chunk[col].astype('category')

  chunk_memory = chunk.memory_usage(deep=True).sum()/(1024*1024)
  memory_footprints.append(chunk_memory)

  chunk.to_sql("investments", conn, if_exists='append', index=False)

query_df = pd.read_sql('PRAGMA table_info(investments);', conn)
print(query_df)

    cid                    name  type  notnull dflt_value  pk
0     0            company_name  TEXT        0       None   0
1     1   company_category_code  TEXT        0       None   0
2     2    company_country_code  TEXT        0       None   0
3     3            company_city  TEXT        0       None   0
4     4           investor_name  TEXT        0       None   0
5     5  investor_category_code  TEXT        0       None   0
6     6   investor_country_code  TEXT        0       None   0
7     7           investor_city  TEXT        0       None   0
8     8      funding_round_type  TEXT        0       None   0
9     9            funded_month  TEXT        0       None   0
10   10          funded_quarter  TEXT        0       None   0
11   11       raised_amount_usd  REAL        0       None   0


## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [84]:
# Your code goes here
query1 = 'select investor_name,raised_amount_usd from investments;'
q1_df = pd.read_sql(query1, conn)
total_investors = len(q1_df['investor_name'].unique())
top_10_investors_num = int(0.1 * total_investors)
top_01_investors_num = int(0.01 * total_investors)
bottom_10_investors_num = int(0.1 * total_investors)
bottom_01_investors_num = int(0.01 * total_investors)
#top_10_investors_amt = q1_df.groupby(["investor_name"])["raised_amount_usd"].nlargest(top_10_investors_num)
total_funds = q1_df['raised_amount_usd'].sum()
investor_sums = q1_df.groupby(['investor_name']).sum().sort_values('raised_amount_usd', ascending=False)
top_10_investors = investor_sums.head(top_10_investors_num)
top_01_investors = investor_sums.head(top_01_investors_num)
bottom_10_investors = investor_sums.tail(bottom_10_investors_num)
bottom_01_investors = investor_sums.tail(bottom_01_investors_num)
top_10_investors_sum = top_10_investors['raised_amount_usd'].sum()
top_01_investors_sum = top_01_investors['raised_amount_usd'].sum()
bottom_10_investors_sum = bottom_10_investors['raised_amount_usd'].sum()
bottom_01_investors_sum = bottom_01_investors['raised_amount_usd'].sum()
top_10_percent_contrib = (top_10_investors_sum / total_funds) * 100
top_01_percent_contrib = (top_01_investors_sum / total_funds) * 100
bottom_10_percent_contrib = (bottom_10_investors_sum / total_funds) * 100
bottom_01_percent_contrib = (bottom_01_investors_sum / total_funds) * 100

# print(total_funds)
# print(top_10_investors_sum)
print(f'Contribution by top 10% investors: {top_10_percent_contrib}%')
print(f'Contribution by top 1% investors: {top_01_percent_contrib}%')
print(f'Contribution by bottom 10% investors: {bottom_10_percent_contrib}%')
print(f'Contribution by bottom 1% investors: {bottom_01_percent_contrib}%')


Contribution by top 10% investors: 82.94210654630179%
Contribution by top 1% investors: 40.45571695111498%
Contribution by bottom 10% investors: 0.003158276575442076%
Contribution by bottom 1% investors: 0.0%


In [50]:
# Which category of company attracted the most investments?
query2 = 'select company_category_code from investments;'
q2_df = pd.read_sql(query2, conn)
company_categorys_counts = q2_df['company_category_code'].value_counts()
print(q2_df[:5])

  company_category_code
0           advertising
1                  news
2             messaging
3              software
4                   web


The category of company attracted the most investments is *advertising*

In [57]:
# Which investor contributed the most money (across all startups)?
query3 = 'select investor_name,raised_amount_usd from investments;'
q3_df = pd.read_sql(query3, conn)
#top_5_investors_amt = q1_df.groupby(["investor_name"])["raised_amount_usd"].nlargest(5)
#total_funds = q1_df['raised_amount_usd'].sum()
top_5_investors = q3_df.groupby(['investor_name']).sum().sort_values('raised_amount_usd', ascending=False).head(5)
print(top_5_investors)

                                  raised_amount_usd
investor_name                                      
Kleiner Perkins Caufield & Byers       2.243565e+10
New Enterprise Associates              1.938508e+10
Accel Partners                         1.294425e+10
Goldman Sachs                          1.275092e+10
Sequoia Capital                        1.207880e+10


**Kleiner Perkins Caufield & Byers** contributed the most money (across all startups)

In [68]:
# Which investors contributed the most money per startup?
query4 = 'select company_name, company_category_code,investor_name,raised_amount_usd from investments;'
q4_df = pd.read_sql(query4, conn)
top_5_investors_per_startup = q4_df.groupby(['investor_name','company_name']).agg({'raised_amount_usd':sum}).sort_values('raised_amount_usd', ascending=False)
print(top_5_investors_per_startup)

                                                                raised_amount_usd
investor_name              company_name                                          
Comcast                    Clearwire                                 1.124000e+10
Intel                      Clearwire                                 1.124000e+10
Time Warner                Clearwire                                 1.124000e+10
BrightHouse                Clearwire                                 9.400000e+09
Google                     Clearwire                                 6.400000e+09
...                                                                           ...
Coriolis Ventures          Martini Media Inc                         0.000000e+00
Cornell University         Flicstart                                 0.000000e+00
Norwest Venture Partners   Amcom Software                            0.000000e+00
Corporate Finance Partners zuuka!                                    0.000000e+00
CustomInk       

*Comcast* contributed the most money per startup (Clearwire).

In [65]:
# Which funding round was the most popular? Which was the least popular?
query5 = 'select funding_round_type from investments;'
q5_df = pd.read_sql(query5, conn)
funding_round_type_counts = q5_df['funding_round_type'].value_counts()
print(funding_round_type_counts)

series-a          27876
series-c+         21740
angel             17978
venture           17834
series-b          17588
other              1928
private-equity      714
post-ipo             66
crowdfunding         10
Name: funding_round_type, dtype: int64


*series-a* was the most popular funding round type while *crowdfunding* was the least ppopular.